# YOUR PROJECT TITLE

# Introduction & Household Optimization
The Diamond Overlapping Generations Model (Diamond-OLG) is a particular class of models employed to examine the dynamics of an economy, especially in relation to economic growth and intergenerational effects. Diamond-OLG models are characterized by distinct time periods in which at least two separate generations coexist. In the version we explore here, each period is comprised of two generations: a young working generation and an older retired generation. Individuals in the model live for two periods, overlapping with the life cycles of others.

During their two-period lives, individuals consume in both periods, but only save in their first period, as they exhaust all their resources before passing away at the end of the second period. Consequently, they earn labor income in their first period and capital income in their second period.

**Beginning of period t:** generation t is born
**Period t:** generation t is employed
**End of period t:** generation t consumes and saves
**Period t+1:** generation t invests
**End of period t+1:** generation t consumes and passes away
First, we will derive the general framework of the Diamond-OLG model, and then we will expand the model by incorporating government intervention, including taxation and debt issuance. By doing so, we demonstrate that the Diamond-OLG model is versatile and capable of addressing more complex issues in economic analysis.

We usemake use of this utility function:



Imports and set magics:

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 